In [1]:
import wandb
wandb.login()

wandb: Currently logged in as: bgcho96 (use `wandb login --relogin` to force relogin)


True

In [2]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [3]:
# classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
#            "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# classes = ("UNKNOWN", "Paper pack", "Metal", "Glass", "Styrofoam", "Battery", "Clothing")
classes = ("UNKNOWN","General trash", "Paper",
           "Plastic", "Plastic bag")
# config file 들고오기
cfg = Config.fromfile('./configs/cascade_rcnn/cascade_rcnn_x101_64x4d_fpn_20e_coco.py')

PREFIX = '../../input/data/'
# for i in range(1,5):
# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.data.samples_per_gpu = 4

cfg.seed=2020
cfg.gpu_ids = [0]
cfg.work_dir = f'./work_dirs/cascade_rcnn_x101_64x4d_fpn_20e_coco_plenty'

# cfg.model.roi_head.bbox_head.num_classes = 11

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)


group_name = 'PSTG'; ##그룹이름(run 여러개를 그룹으로 묶어줄 수 있습니다)
project_name = 'mmdet'; ##프로젝트 이름
run_name = f'4class_plenty5'; ##run 이름
config_list = {
    'epoch' : cfg.runner.max_epochs,
    'batch_size' :  cfg.data.samples_per_gpu,
    'optimizer' : cfg.optimizer,
    'optimizer_config' : cfg.optimizer_config,
    'lr_config' : cfg.lr_config
}
## 원하는 wandb.init() argument값 추가해주세요
cfg.log_config.hooks[1].init_kwargs['group']=group_name   # group name(option)
cfg.log_config.hooks[1].init_kwargs['name'] = run_name    # run name
cfg.log_config.hooks[1].init_kwargs['config'] = config_list    # config

model = build_detector(cfg.model)

datasets = [build_dataset(cfg.data.train)]

2021-05-19 19:14:01,174 - mmdet - INFO - load model from: open-mmlab://resnext101_64x4d
2021-05-19 19:14:01,176 - mmdet - INFO - Use load_from_openmmlab loader


loading annotations into memory...
Done (t=3.80s)
creating index...
index created!


In [4]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...


2021-05-19 19:14:10,412 - mmdet - INFO - Start running, host: root@e6ef85cb398c, work_dir: /opt/ml/code/mmdetection_trash/work_dirs/cascade_rcnn_x101_64x4d_fpn_20e_coco_plenty
2021-05-19 19:14:10,413 - mmdet - INFO - workflow: [('train', 1)], max: 15 epochs


Done (t=1.37s)
creating index...
index created!


/opt/ml/code/mmdetection_trash/mmdet/core/evaluation/eval_hooks.py:111: UserWarning: runner.meta is None. Creating a empty one.
  warnings.warn('runner.meta is None. Creating a empty one.')


2021-05-19 19:15:05,606 - mmdet - INFO - Epoch [1][50/601]	lr: 1.978e-03, eta: 2:28:52, time: 0.996, data_time: 0.052, memory: 5810, loss_rpn_cls: 0.4912, loss_rpn_bbox: 0.0627, s0.loss_cls: 0.6560, s0.acc: 83.2383, s0.loss_bbox: 0.1520, s1.loss_cls: 0.2193, s1.acc: 94.8057, s1.loss_bbox: 0.0427, s2.loss_cls: 0.1182, s2.acc: 89.4990, s2.loss_bbox: 0.0060, loss: 1.7481, grad_norm: 8.5658
2021-05-19 19:15:53,453 - mmdet - INFO - Epoch [1][100/601]	lr: 3.976e-03, eta: 2:25:07, time: 0.957, data_time: 0.008, memory: 5810, loss_rpn_cls: 0.2005, loss_rpn_bbox: 0.0560, s0.loss_cls: 0.3807, s0.acc: 90.3633, s0.loss_bbox: 0.2297, s1.loss_cls: 0.1078, s1.acc: 95.7793, s1.loss_bbox: 0.0682, s2.loss_cls: 0.0328, s2.acc: 97.8936, s2.loss_bbox: 0.0097, loss: 1.0855, grad_norm: 3.6942
2021-05-19 19:16:41,021 - mmdet - INFO - Epoch [1][150/601]	lr: 5.974e-03, eta: 2:23:03, time: 0.951, data_time: 0.008, memory: 5810, loss_rpn_cls: 0.1980, loss_rpn_bbox: 0.0591, s0.loss_cls: 0.4106, s0.acc: 88.9404, s0

...

Loading and preparing results...
DONE (t=0.64s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=4.61s).
Accumulating evaluation results...


2021-05-19 21:59:20,726 - mmdet - INFO - Epoch(val) [15][601]	bbox_mAP: 0.1700, bbox_mAP_50: 0.2590, bbox_mAP_75: 0.1800, bbox_mAP_s: 0.0340, bbox_mAP_m: 0.1490, bbox_mAP_l: 0.3230, bbox_mAP_copypaste: 0.170 0.259 0.180 0.034 0.149 0.323


DONE (t=0.68s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.170
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.259
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.180
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.034
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.149
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.323
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.303
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.303
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.303
Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.119
Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.286
Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.545


train/loss_rpn_cls,0.03883
train/loss_rpn_bbox,0.03226
train/s0.loss_cls,0.20954
train/s0.acc,92.7627
train/s0.loss_bbox,0.10815
train/s1.loss_cls,0.09618
train/s1.acc,93.42971
train/s1.loss_bbox,0.10602
train/s2.loss_cls,0.04779
train/s2.acc,93.50487
train/s2.loss_bbox,0.06747


train/loss_rpn_cls,█▇▅▅▄▃▃▃▃▃▃▃▃▃▃▂▂▃▃▂▂▂▂▂▂▂▂▂▂▁▁▂▁▂▁▁▁▁▁▁
train/loss_rpn_bbox,▆█▆▆▆▄▃▃▃▄▃▃▃▄▄▃▂▄▄▃▄▂▃▂▄▃▁▃▁▂▁▂▂▃▂▁▂▁▁▂
train/s0.loss_cls,▇█▇▆▆▅▄▅▅▄▅▅▄▅▅▄▃▄▄▄▄▃▄▃▃▂▂▃▂▂▂▃▂▂▂▁▂▂▁▂
train/s0.acc,▃▁▁▃▂▄▅▂▃▄▃▃▄▃▃▄▅▄▄▅▅▆▄▅▅▆▆▅▇▆▆▅▇▇▆▇▆▇█▇
train/s0.loss_bbox,▇█▇▅▅▄▃▄▄▄▄▄▃▄▄▃▃▃▃▃▃▂▃▂▃▂▂▃▁▂▂▂▁▂▂▁▂▁▁▁
train/s1.loss_cls,▃████▆▅▇▇▆▇▆▆▇▇▆▄▆▆▅▅▄▅▄▄▃▃▄▂▃▃▄▂▂▃▁▃▂▁▂
train/s1.acc,█▃▁▂▁▃▃▁▂▂▁▂▂▂▂▃▃▂▂▃▃▄▃▄▄▅▅▃▅▅▅▄▅▆▄▆▄▆▆▅
train/s1.loss_bbox,▁▆█▇▇▆▅▆▆▆▆▆▅▆▆▅▅▅▅▅▅▄▅▄▅▄▃▅▃▄▄▄▃▃▄▃▄▃▃▄
train/s2.loss_cls,▁▅▇▇█▆▆█▇▆█▇▇█▇▇▆▇▇▆▆▅▆▆▅▅▄▆▃▄▄▅▄▄▅▃▄▃▃▄
train/s2.acc,█▅▃▂▂▃▃▁▂▂▁▂▂▂▂▂▃▂▂▂▂▃▃▃▃▃▄▂▄▄▄▃▄▄▃▄▄▄▅▄
train/s2.loss_bbox,▁▄▇██▇▇█▇▇██▇██▇█▇▇▇▇▇▇▇▇▇▆▇▆▇▇▇▆▇▇▆▇▆▆▇
